Not reviewed also
```sql
select t.transcript, t.reviewed_transcript, t.final_transcript, t.file_name, t.url 
from "Task" t 
where t.state in ('submitted', 'accepted', 'finalised');
```
reviewed at least once
```sql 
select t.reviewed_transcript, t.final_transcript, t.file_name, t.url 
from "Task" t 
where t.state in ('accepted', 'finalised');
```

In [ ]:
! ls /home/monlamai

In [ ]:
! rm /home/monlamai/_Task__202401081531.csv

In [5]:
csv_path = "/home/monlamai/_Task__202401171255.csv"

In [ ]:
! pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv(csv_path)

In [7]:
df.head()

,reviewed_transcript,final_transcript,file_name,url
0,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,NaN,STT_AB00237_0190_1711732_to_1723319,https://huggingface.co/datasets/openpecha/STT_...
1,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,NaN,STT_AB00237_0197_1767500_to_1776441,https://huggingface.co/datasets/openpecha/STT_...
2,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,NaN,STT_AB00238_0005_30554_to_39889,https://huggingface.co/datasets/openpecha/STT_...
3,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,NaN,STT_AB00238_0006_41271_to_46288,https://huggingface.co/datasets/openpecha/STT_...
4,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,NaN,STT_AB00238_0007_47056_to_49957,https://huggingface.co/datasets/openpecha/STT_...


In [8]:
df['grade'] = 1

In [9]:
df.loc[~df['reviewed_transcript'].isna(), 'grade'] = 2

In [10]:
df.loc[~df['final_transcript'].isna(), 'grade'] = 3

In [11]:
df['final_transcript'].fillna(df['reviewed_transcript'], inplace=True)

In [12]:
df.head()

,reviewed_transcript,final_transcript,file_name,url,grade
0,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,STT_AB00237_0190_1711732_to_1723319,https://huggingface.co/datasets/openpecha/STT_...,2
1,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,STT_AB00237_0197_1767500_to_1776441,https://huggingface.co/datasets/openpecha/STT_...,2
2,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,STT_AB00238_0005_30554_to_39889,https://huggingface.co/datasets/openpecha/STT_...,2
3,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,STT_AB00238_0006_41271_to_46288,https://huggingface.co/datasets/openpecha/STT_...,2
4,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,STT_AB00238_0007_47056_to_49957,https://huggingface.co/datasets/openpecha/STT_...,2


In [13]:
df = df[~df['final_transcript'].isnull()]

In [14]:
df['dept'] = df['file_name'].str[:6]

In [15]:
df = df[['file_name','final_transcript', 'url', 'dept', 'grade']]

In [16]:
df.rename(columns={'final_transcript':'uni'}, inplace=True)

In [17]:
df.head()

,file_name,uni,url,dept,grade
0,STT_AB00237_0190_1711732_to_1723319,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
1,STT_AB00237_0197_1767500_to_1776441,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
2,STT_AB00238_0005_30554_to_39889,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
3,STT_AB00238_0006_41271_to_46288,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
4,STT_AB00238_0007_47056_to_49957,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2


In [18]:
df.to_csv('01_stt_pecha_tools.tsv', sep='\t', index=False)

In [19]:
df.shape

(168277, 5)